In [ ]:
from openpyxl import load_workbook
from openpyxl.utils.cell import coordinate_from_string, column_index_from_string, get_column_letter
import pandas as pd

from loading import *

# Load the workbook, data_only=True to get the values of the formulas not the formulas themselves
wb = load_workbook('SAMPLES\zz_lista_volontera_24_25.xlsx', data_only=True)

sheets = wb.sheetnames

<>:8: SyntaxWarning: invalid escape sequence '\z'
<>:8: SyntaxWarning: invalid escape sequence '\z'
C:\Users\Lobel\AppData\Local\Temp\ipykernel_20764\2588155385.py:8: SyntaxWarning: invalid escape sequence '\z'
  wb = load_workbook('SAMPLES\zz_lista_volontera_24_25.xlsx', data_only=True)


In [19]:
location_dfs = {}

for sheet_name in sheets[1:]:
    
    sheet = wb[sheet_name]
    location_name = location_dict[sheet_name]
    
    this_location_dict = {
        'location_name': [],
        'volunteer_name': [],
        'volunteer_dates': [],
        'volunteer_class': [],
        'volunteer_school': []
    }
    
    schools = load_all_schools(sheet)
    
    volunteer_df = pd.DataFrame(this_location_dict)
    
    for school_tuple in schools:
        
        school_name, school_range = school_tuple
        
        if schools.index((school_name, school_range)) == len(schools)-1:
            next_school_start_row = sheet.max_row+1
        else:
            next_school_start_row = int(coordinate_from_string(schools[schools.index((school_name, school_range))+1][1][0])[1])
            
        classes = load_classes(sheet, school_tuple, next_school_start_row)
        
        for class_tuple in classes:
            
            if classes.index(class_tuple) == len(classes)-1:
                next_class_start_row = next_school_start_row
            else:
                next_class_start_row = int(classes[classes.index(class_tuple)+1][1][1:])
                
            class_volunteer_df = load_volunteers(sheet, location_name, school_name, class_tuple, next_class_start_row)
            
            volunteer_df = pd.concat([volunteer_df, class_volunteer_df], ignore_index=True)
            
    location_dfs[location_name] = volunteer_df
    
# for i in location_dfs: display(location_dfs[i])
    
merged_list = [i['volunteer_dates'].sum() for i in location_dfs.values()]

merged_list = [item for sublist in merged_list for item in sublist]

print(len(merged_list))

#k = list(location_dfs.values())[2]
#count = k[k['volunteer_dates'].apply(lambda x: x == [])].shape[0]
#print(len(k))
#print(count)


327


In [15]:
sheet = wb[sheets[3]]

# schools = load_all_schools(sheet)

# print(schools)
    

classes = load_classes(sheet, schools[0], coordinate_from_string(schools[1][1][0])[1])

print(classes)

# volunteers = load_volunteers(sheet, 'XV.', ('3. razred', 'C19'), 33)

# display(volunteers)

[('1. razred', 'C12'), ('2. razred', 'C15'), ('3. razred', 'C19'), ('4. razred', 'C33')]


In [16]:
for i in volunteers['volunteer_hours']:
    for j in i:
        print(i, j.value)

NameError: name 'volunteers' is not defined

In [ ]:
column_index_from_string(coordinate_from_string('B2')[0])